In [7]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

from dotenv import load_dotenv
load_dotenv(override=True)

ROOT_PATH = os.environ.get('ROOT_PATH')
if not ROOT_PATH in sys.path: sys.path.append(ROOT_PATH)

from main_code.utils.constants import *
from main_code.utils.CosmoFunc import *
from main_code.utils.logging_config import get_logger
from main_code.utils.filepaths import (
    LOGDIST_OUTPUT_FILEPATH
)
from main_code.step_12_smoothed_field import generate_grids, get_supergalactic_coords

from astropy import units as u
from astropy.coordinates import SkyCoord

SMIN_SETTING = int(os.environ.get('SMIN_SETTING'))
FP_FIT_METHOD = int(os.environ.get('FP_FIT_METHOD'))

# Calculate CF4 logdists

Doesn't seem feasible since for some reason they only provided `Vcmb`.

In [2]:
# Load CF4
df = pd.read_csv("../../data/external/cf4/raw/CF4_all_individual.txt")

df

,pgc,1PGC,T17,Vcmb,DM,eDM,DMsnIa,eDMsn1,DMtf,eDMtf,...,eDMmas,RA,DE,glon,glat,sgl,sgb,SGX,SGY,SGZ
0,2,73150,0,4726,34.523,0.46,NaN,NaN,34.523,0.46,...,NaN,0.0070,47.2745,113.9553,-14.6992,341.6440,20.7388,41.80,-13.87,16.68
1,4,120,0,4109,33.483,0.39,NaN,NaN,33.483,0.39,...,NaN,0.0144,23.0876,107.8322,-38.2729,316.0587,18.4514,27.98,-26.96,12.96
2,12,12,0,6195,34.983,0.41,NaN,NaN,34.983,0.41,...,NaN,0.0358,-6.3739,90.1920,-65.9300,286.4249,11.3511,17.09,-57.99,12.14
3,16,16,0,5312,34.643,0.45,NaN,NaN,34.643,0.45,...,NaN,0.0471,-5.1587,91.6006,-64.8655,287.6120,11.7030,15.68,-49.38,10.73
4,35,35,0,34,29.730,0.10,NaN,NaN,NaN,NaN,...,NaN,0.0938,39.4955,112.3102,-22.3207,333.3464,20.3429,0.28,-0.14,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55869,9003283,9003283,0,7034,34.912,0.10,34.912,0.10,NaN,NaN,...,NaN,341.2630,8.9422,78.1768,-42.6849,296.8603,33.5664,26.34,-52.01,38.68
55870,9003285,71884,0,16166,36.842,0.10,36.842,0.10,NaN,NaN,...,NaN,354.3419,19.9315,100.1204,-39.6369,311.8106,23.1462,97.88,-109.44,62.77
55871,9003286,9003286,0,25533,37.812,0.10,37.812,0.10,NaN,NaN,...,NaN,340.4511,17.6285,84.1818,-35.3097,307.1052,35.8547,122.41,-161.83,146.64
55872,9003287,9003287,0,23881,37.512,0.11,37.512,0.11,NaN,NaN,...,NaN,131.4750,43.8416,176.9949,38.5558,48.2306,-25.2828,141.22,158.12,-100.14


# Calculate velocity field prediction from grids

pvhub takes CMB redshift as argument, while my grid returns comoving coordinates

In [4]:
grid_coords = generate_grids()

grid_coords

array([[-180.,  -32.,  -80.],
       [-180.,  -28.,  -80.],
       [-176.,  -48.,  -80.],
       ...,
       [ 172.,  -72.,   70.],
       [ 176.,  -64.,   70.],
       [ 176.,  -60.,   70.]])

In [ ]:
pv_model = TwoMPP_SDSS_6dF(verbose=True) # type: ignore

# Calculate predicted PVs using observed group redshift in CMB frame, and calculate cosmological redshift
df['v_pec'] = pv_model.calculate_pv(df['ra'].to_numpy(), df['dec'].to_numpy(), df['z_dist_est'].to_numpy())

# Position of the nearby superclusters

In [ ]:
sc_list = [
    {
        'name': 'perseus-pisces',
        'z': 0.03,
        'ra_hh': 1,
        'ra_mm': 50,
        'ra_ss': 0,
        'dec_dd': 36,
        'dec_mm': 0,
        'dec_ss': 0,
        'slice': 1,
        'marker': 'v'
    },
    {
        'name': 'coma',
        'z': 0.023,
        'ra_hh': 12,
        'ra_mm': 24,
        'ra_ss': 6.8,
        'dec_dd': 23,
        'dec_mm': 55,
        'dec_ss': 23,
        'slice': 1,
        'marker': '^'
    },
    {
        'name': 'sculptor wall',
        'z': 0.03,
        'ra_hh': 0,
        'ra_mm': 0,
        'ra_ss': 0,
        'dec_dd': -30,
        'dec_mm': 0,
        'dec_ss': 0,
        'slice': 1,
        'marker': 'p'
    },
    {
        'name': 'hercules',
        'z': 0.035,
        'ra_hh': 16,
        'ra_mm': 5,
        'ra_ss': 15,
        'dec_dd': 17,
        'dec_mm': 44,
        'dec_ss': 55,
        'slice': 3,
        'marker': '>'
    },
    {
        'name': 'leo',
        'z': 0.0314,
        'ra_hh': 11,
        'ra_mm': 10,
        'ra_ss': 31.4,
        'dec_dd': 28,
        'dec_mm': 43,
        'dec_ss': 39,
        'slice': 0,
        'marker': 'o'
    },
    {
        'name': 'ophiuchus',
        'z': 0.028,
        'ra_hh': 17,
        'ra_mm': 10,
        'ra_ss': 0,
        'dec_dd': -22,
        'dec_mm': 0,
        'dec_ss': 0,
        'slice': 2,
        'marker': 'h'
    },
    {
        'name': 'shapley',
        'z': 0.046,
        'ra_hh': 13,
        'ra_mm': 25,
        'ra_ss': 0,
        'dec_dd': -30,
        'dec_mm': 0,
        'dec_ss': 0,
        'slice': 1,
        'marker': 'D'
    },
    {
        'name': 'pavo-indus',
        'z': 0.015,
        'ra_hh': 20,
        'ra_mm': 48,
        'ra_ss': 26,
        'dec_dd': -39,
        'dec_mm': 6,
        'dec_ss': 9,
        'slice': 0,
        'marker': 'D'
    }
]

# Calculate ra dec
df = pd.DataFrame(sc_list)
df['ra'] = (df['ra_hh'] + df['ra_mm']/60 + df['ra_ss']/3600) * 15
df['dec'] = df['dec_dd'] + df['dec_mm']/60 + df['dec_ss']/3600

# Get supergalactic coordinates
df = get_supergalactic_coords(df, z_dist_est_col_name='z')

df.to_csv("./supercluster_list.csv", index=False)

df

,name,z,ra_hh,ra_mm,ra_ss,dec_dd,dec_mm,dec_ss,slice,marker,ra,dec,SGLON,SGLAT,dz_dist_est,SGX,SGY,SGZ
0,perseus-pisces,0.0300,1,50,0.0,36,0,0,1,v,27.500000,36.000000,334.626883,-1.484851,89.302559,80.661023,-38.254301,-2.314064
1,coma,0.0230,12,24,6.8,23,55,23,1,^,186.028333,23.923056,91.385131,-0.487082,68.579402,-1.657693,68.556885,-0.582999
2,sculptor wall,0.0300,0,0,0.0,-30,0,0,1,p,0.000000,-30.000000,263.893475,3.602625,89.302559,-9.480993,-88.620368,5.611437
3,hercules,0.0350,16,5,15.0,17,44,55,3,>,241.312500,17.748611,107.331736,49.387156,104.062032,-20.179576,64.662991,78.996132
4,leo,0.0314,11,10,31.4,28,43,39,0,o,167.630833,28.727500,80.679760,-13.978218,93.438810,14.684528,89.474867,-22.570426
5,ophiuchus,0.0280,17,10,0.0,-22,0,0,2,h,257.500000,-22.000000,171.450793,44.351114,83.388779,-58.966201,8.864337,58.293217
6,shapley,0.0460,13,25,0.0,-30,0,0,1,D,201.250000,-30.000000,147.253810,-1.525237,136.406911,-114.687792,73.758885,-3.630778
7,pavo-indus,0.0150,20,48,26.0,-39,6,9,1,D,312.108333,-38.897500,231.526260,29.295011,44.810517,-24.313717,-30.595304,21.926078


In [ ]:
sgz_grid_list = [-20., 0., 60., 80.]

df

,name,z,ra_hh,ra_mm,ra_ss,dec_dd,dec_mm,dec_ss,ra,dec,SGLON,SGLAT,dz_dist_est,SGX,SGY,SGZ
0,perseus-pisces,0.0300,1,50,0.0,36,0,0,27.500000,36.000000,334.626883,-1.484851,89.302559,80.661023,-38.254301,-2.314064
1,coma,0.0230,12,24,6.8,23,55,23,186.028333,23.923056,91.385131,-0.487082,68.579402,-1.657693,68.556885,-0.582999
2,sculptor wall,0.0300,0,0,0.0,-30,0,0,0.000000,-30.000000,263.893475,3.602625,89.302559,-9.480993,-88.620368,5.611437
3,hercules,0.0350,16,5,15.0,17,44,55,241.312500,17.748611,107.331736,49.387156,104.062032,-20.179576,64.662991,78.996132
4,leo,0.0314,11,10,31.4,28,43,39,167.630833,28.727500,80.679760,-13.978218,93.438810,14.684528,89.474867,-22.570426
5,ophiuchus,0.0280,17,10,0.0,-22,0,0,257.500000,-22.000000,171.450793,44.351114,83.388779,-58.966201,8.864337,58.293217
6,shapley,0.0460,13,25,0.0,-30,0,0,201.250000,-30.000000,147.253810,-1.525237,136.406911,-114.687792,73.758885,-3.630778
